In [25]:
%%html
<style>
table {float:left}
</style>

In [26]:
import numpy as np
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
pd.set_option('display.max_rows', 1000)

In [27]:
STABLE_COINS   = ['USDT', 'USDC']
UNSTABLE_COINS = ['BTC', 'ETH']
JOIN_WINDOW_MINUTES = 30 
DOLLAR_MIN = 100000000

In [28]:
def prep_prices_df(prices_df):
    """
    Fiter to just unstable coins (BTC/ETH), and rename columns
    """
    prices_df = prices_df[prices_df['asset'].isin(UNSTABLE_COINS)]
        
    prices_df = prices_df.rename(columns={
        'mid': 'price', 
        'unix_time': 'price_time', 
        'asset': 'price_asset'
    })
    
    prices_df = prices_df[['price_asset', 'price', 'price_time']]
    
    return prices_df

def prep_tweets_df(tweets_df, dollar_min):
    """
    Round tweet to nearest minute and filter for only transactions about {dollar_min} dollars
    """
    def _round_to_minute(tweet_time):
        sec_after_min = tweet_time % 60
        return tweet_time - sec_after_min if sec_after_min <= 30 else tweet_time - sec_after_min + 60
    
    tweets_df['tweet_time'] = tweets_df['unix_time'].apply(_round_to_minute)
    
    tweets_df = tweets_df[(tweets_df['usd_value'] >= dollar_min)]
    
    tweets_df = tweets_df.rename(columns={'asset': 'tweet_asset'})
    
    return tweets_df

In [29]:
def subset_tweets(tweets_df, transfer_type, coin_type, source, recipient):
    """
    Filters the twitter dataframe to the given transfer type
    """
            
    is_stable_coin   = tweets_df['tweet_asset'].isin(STABLE_COINS)
    is_unstable_coin = tweets_df['tweet_asset'].isin(UNSTABLE_COINS)
    
    coin_cond = is_stable_coin if coin_type == 'stable' else is_unstable_coin

    transfer_type_col = tweets_df['transaction_type']
    source_col        = tweets_df['transaction_source']
    recipient_col     = tweets_df['transaction_recipient']
    
    # If it was a mint, all we need to filter for is the transaction type
    if transfer_type == 'MINTED':
        return tweets_df[(is_stable_coin) & (transfer_type_col == 'MINTED')]
    
    # Otherwise, filter on coin type, transfer type, source and recipient
    else:
        filter_stmt = (
            (coin_cond) 
            & (transfer_type_col == transfer_type) 
            & (source_col == source) 
            & (recipient_col == recipient)
        ) 
        return tweets_df[filter_stmt]

In [30]:
def join_tweets_and_prices(tweets_df, prices_df, coin_type):
    """
    Joins prices within {JOIN_WINDOW_MINUTES} of the tweet
    """
    
    # We'll take the cartesian product for the join and then filter
    df = tweets_df.merge(prices_df, how='cross')

    # For the plots from unstable coin tweets, 
    #  we'll only want to plot the price change for the corresponding coin
    #  i.e. we only want to plot BTC price changes from BTC tweets
    #
    # For plots about stable coins, we'll plot both BTC and ETH
    if coin_type == 'unstable':
        df = df[(df['tweet_asset'] == df['price_asset'])]

    # Filter to just prices within the specified time window of the tweet 
    time_filter = (
        (df['price_time'] >= (df['tweet_time'] - (JOIN_WINDOW_MINUTES * 60)))
        & (df['price_time'] <= (df['tweet_time'] + (JOIN_WINDOW_MINUTES * 60)))
    )

    df = df[time_filter]
        
    return df

In [31]:
def get_average_price_change(df):
    """
    Get the weighted average price change at each minute interval surrounding the tweet
    """
    # Get time difference between tweet and price in minutes
    df['time_delta'] = ((df['price_time'] - df['tweet_time']) / 60).astype('int')
    
    # Get a dataframe with just the starting prices
    start_prices_df = df[df['time_delta'] == 0]
    start_prices_df = start_prices_df.rename(columns={'price': 'start_price'})
    start_prices_df = start_prices_df[['price_asset', 'start_price', 'tweet_time']]
    
    # Join the t=0 price back to the original dataframe
    df = df.merge(start_prices_df, how='inner', on=['tweet_time', 'price_asset'])
    
    # Calculate percent price change
    df['price_change'] = ((df['price'] - df['start_price']) / df['start_price']) * 100
    
    # Get the weighted average price change
    # Weighted by transaction size in dollars
    df = (df
          .groupby(['price_asset', 'time_delta'])
          .apply(lambda df: np.average(df['price_change'], weights=df['usd_value']))
          .reset_index(name='average_price_change'))
    
    return df

In [32]:
def get_price_plots(df, title):
    """
    Returns a plot of the average price change for both BTC and ETH surrounding a whale alert
    Input dataframe has 1 record per asset/time_delta combination
    """
    figs = []
        
    for asset in UNSTABLE_COINS:
        
        asset_df = df[df['price_asset'] == asset]
        
        rolling_window = 4
        rolling_name = f'{rolling_window}_min_rolling_average'
        asset_df = asset_df.assign(
            rolling=asset_df['average_price_change'].rolling(rolling_window).mean().shift(-1 * (rolling_window // 2))
        )
        asset_df = asset_df.rename(columns={'rolling': rolling_name})
        
        fig = px.line(asset_df, 
                      x='time_delta', 
                      y=['average_price_change', rolling_name] , 
                      color_discrete_sequence=['rgba(57,118,175, 0.2)', 'rgb(57,118,175)'],
                      title=title.format(asset))

        fig.add_vline(x=0, line_color='lightgrey', line_width=3)
        fig.add_hline(y=0, line_color='lightgrey', line_width=3)
        
        fig.update_layout(yaxis_range=[-2, 2])
        fig.update_yaxes(tickvals=list(np.linspace(-2, 2, 9)))
        
        fig.update_layout(height=400, width=1200)
        
        figs.append(fig)
    
    return figs

In [33]:
def get_plot_title(transfer_type, tweet_coin_type, source, recipient, num_tweets):

    def _get_non_mint_event():
        coins = STABLE_COINS if tweet_coin_type == 'stable' else UNSTABLE_COINS
        coins_str = '/'.join(coins)
        
        return f'Transfer of <b>{coins_str}</b> from <b>{source.capitalize()}</b> to <b>{recipient.capitalize()}</b>'

    event = 'USDC/USDT Minting' if transfer_type == 'MINTED' else _get_non_mint_event()
        
    title = f'Weighted Price Change of <b>{{}}</b> surrounding {event} <br>'
    title += f'Composed of {num_tweets} tweets'
    
    return title
    

| Direction          |     Option Name    | Event                                        |
|--------------------|:-------------------|:---------------------------------------------|
| **Minting**        |       `mint`       | USDT/USDC Minting                            |
| **Bullish**        | `mint_to_exchange` | USDT/USDC Transfer from treasury to exchange |
| **Bullish**        |    `sell_stable`   | USDT/USDC Transfer from wallet to exchange   |
| **Bearish**        | `sell_unstable`    | BTC/ETH Transfer from wallet to exchange     |

In [34]:
prices_df = pd.read_csv('data/prices_formatted.csv')
tweets_df = pd.read_csv('data/tweets_formatted.csv')

tweets_df = prep_tweets_df(tweets_df, DOLLAR_MIN)
prices_df = prep_prices_df(prices_df)

plots = [
    ('Minting',                               'MINTED',   'stable',   None,        None),
    ('Bullish - Stable Treasury to Exchange', 'TRANSFER', 'stable',   'TREASURY', 'EXCHANGE'),
    ('Bullish - Stable Wallet to Exchange',   'TRANSFER', 'stable',   'WALLET',   'EXCHANGE'),
    ('Bearish - BTC/ETH Wallet to Exchange',  'TRANSFER', 'unstable', 'WALLET',   'EXCHANGE'),
]

plot_file = 'plots/average_price_impact.html'        
html_file = open(plot_file, 'w')

heading = f'Weighted Average Price Change Surrounding Whale Alerts - ${DOLLAR_MIN:,} Minimum'
html_file.write(f'<h1 style="font-family:arial">{heading}<h1>')

figs = []
for (direction, transfer_type, tweet_coin_type, source, recipient) in plots:
    
    html_file.write(f'<h2 style="font-family:arial">{direction}<h2>')
    
    example_tweet_df = subset_tweets(tweets_df, transfer_type, tweet_coin_type, source, recipient)
    num_tweets = example_tweet_df.shape[0]
    
    joined_df = join_tweets_and_prices(example_tweet_df, prices_df, tweet_coin_type)
    joined_df = get_average_price_change(joined_df)
    
    title = get_plot_title(transfer_type, tweet_coin_type, source, recipient, num_tweets)
    
    [html_file.write(fig.to_html()) for fig in get_price_plots(joined_df, title)]
     

html_file.close()